In [127]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [128]:
post = pd.read_csv('lv2-2402.csv')
post.shape

(7488, 7)

# 1.

## 1-1.

In [129]:
df1 = post.copy()

In [130]:
yes_map = {'heart emoji': 5, 'thumbs up': 4, 'clap': 3, 'laughing': 2, np.nan: 1, 'fire': 0}
df1.loc[df1['LIKES'] == 'yes', 'LIKE_SCORE'] = df1.loc[df1['LIKES'] == 'yes', 'LIKE_TYPE'].map(yes_map)

In [131]:
no_map = {'fire': -5, np.nan: -1}
df1.loc[df1['LIKES'] == 'no', 'LIKE_SCORE'] = df1.loc[df1['LIKES'] == 'no', 'LIKE_TYPE'].map(no_map).fillna(0)

In [132]:
df1.head()

,PID,USER_ID,POST_ID,CREATED_DATE,LIKES,NUM_HASHTAG,LIKE_TYPE,LIKE_SCORE
0,1,2,1,2023-04-13,yes,1,heart emoji,5.0
1,2,3,1,2023-04-13,no,2,NaN,-1.0
2,3,5,1,2023-04-13,no,4,fire,-5.0
3,4,14,1,2023-04-13,yes,2,laughing,2.0
4,5,17,1,2023-04-13,yes,1,NaN,1.0


## 1-2.

In [133]:
AGG24 = df1.groupby('POST_ID').apply(lambda x: pd.Series({'NUM_LIKES': (x['LIKES'] == 'yes').sum(),
                                                          'NUM_USER': x['USER_ID'].nunique(),
                                                          'SUM_SCORE': x['LIKE_SCORE'].sum()}))
AGG24

,NUM_LIKES,NUM_USER,SUM_SCORE
POST_ID,,,
1,13.0,25.0,0.0
2,15.0,31.0,5.0
3,14.0,27.0,23.0
4,16.0,32.0,22.0
5,13.0,27.0,18.0
...,...,...,...
253,13.0,26.0,31.0
254,13.0,26.0,26.0
255,15.0,29.0,32.0


## 1-3.

In [134]:
AGG24['POPULAR'] = AGG24['NUM_LIKES'] + (AGG24['SUM_SCORE'] / AGG24['NUM_USER'])

In [135]:
AGG24['POPULAR'].max().round(2)

20.82

# 2.

## 2-1.

In [136]:
df2 = post.copy()

In [147]:
# 참고
test = [False, True, True]
np.sum(test)

2

In [71]:
AGG25 = df2.groupby('USER_ID')[['POST_ID', 'LIKES']]\
            .apply(lambda x: pd.Series({'NUM_POST': x['POST_ID'].nunique(),
                                        'NUM_LIKE': (x['LIKES'] == 'yes').sum()}))
AGG25

,NUM_POST,NUM_LIKE
USER_ID,,
2,66,29
3,67,25
4,64,32
5,257,135
6,62,31
...,...,...
96,75,42
97,68,41
98,65,28


## 2-2.

In [73]:
from sklearn.preprocessing import StandardScaler

In [74]:
cols = AGG25.columns
cols

Index(['NUM_POST', 'NUM_LIKE'], dtype='object')

In [75]:
scaler = StandardScaler().fit(AGG25)

In [78]:
df_scaled = pd.DataFrame(scaler.transform(AGG25), columns=[f'{col}_S' for col in cols])

## 2-3.

In [79]:
from sklearn.cluster import KMeans

In [80]:
model = KMeans(n_clusters=2, random_state=1234).fit(df_scaled)

In [82]:
cluster = model.predict(df_scaled)

In [84]:
AGG25['cluster'] = cluster

## 2-4.

In [85]:
AGG25['cluster'].value_counts()

0    64
1    13
Name: cluster, dtype: int64

In [86]:
Cluster1 = 0
Cluster2 = 1

## 2-5.

In [91]:
df_merge = pd.merge(post, AGG25, left_on='USER_ID', right_index=True, how='left')
df_merge.head()

,PID,USER_ID,POST_ID,CREATED_DATE,LIKES,NUM_HASHTAG,LIKE_TYPE,NUM_POST,NUM_LIKE,cluster
0,1,2,1,2023-04-13,yes,1,heart emoji,66,29,0
1,2,3,1,2023-04-13,no,2,NaN,67,25,0
2,3,5,1,2023-04-13,no,4,fire,257,135,1
3,4,14,1,2023-04-13,yes,2,laughing,257,148,1
4,5,17,1,2023-04-13,yes,1,NaN,83,36,0


In [93]:
Avg_A = df_merge[df_merge['cluster'] == Cluster1]['NUM_HASHTAG'].mean()

## 2-6.

In [95]:
Avg_B = df_merge[df_merge['cluster'] == Cluster2]['NUM_HASHTAG'].mean()

In [96]:
round(abs(Avg_A - Avg_B), 3)

0.026

# 3.

## 3-1.

In [98]:
AGG26 = post.groupby(['POST_ID', 'CREATED_DATE'])[['USER_ID', 'LIKES', 'NUM_HASHTAG']]\
            .apply(lambda x: pd.Series({'NUM_USER': x['USER_ID'].nunique(),
                                        'NUM_LIKES': (x['LIKES'] == 'yes').sum(),
                                        'AVG_HASHTAG': x['NUM_HASHTAG'].mean()}))

In [99]:
AGG26

,,NUM_USER,NUM_LIKES,AVG_HASHTAG
POST_ID,CREATED_DATE,,,
1,2023-04-13,25.0,13.0,2.360000
2,2023-04-13,31.0,15.0,2.387097
3,2023-04-13,27.0,14.0,2.481481
4,2023-04-13,32.0,16.0,2.281250
5,2023-04-13,27.0,13.0,2.629630
...,...,...,...,...
253,2023-04-21,26.0,13.0,2.384615
254,2023-04-21,26.0,13.0,2.346154
255,2023-04-21,29.0,15.0,2.310345


## 3-2.

In [101]:
AGG26['POPULAR'] = np.where(AGG26['NUM_USER'] >= 31, 1, 0)

## 3-3.

In [102]:
from datetime import datetime

In [105]:
AGG26 = AGG26.reset_index()

In [108]:
AGG26['WEEKDAY'] = AGG26['CREATED_DATE'].map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

## 3-4.

In [109]:
DataSet6 = AGG26.copy()

In [110]:
from sklearn.naive_bayes import GaussianNB

In [112]:
DataSet6.columns

cols = ['NUM_LIKES', 'AVG_HASHTAG', 'WEEKDAY']

In [114]:
model = GaussianNB().fit(X=DataSet6[cols], y=DataSet6['POPULAR'])

## 3-5.

In [116]:
tmp = pd.DataFrame({'NUM_LIKES': [16],
                    'AVG_HASHTAG': [2.42],
                    'WEEKDAY': [5]})
tmp

,NUM_LIKES,AVG_HASHTAG,WEEKDAY
0,16,2.42,5


In [126]:
round(model.predict_proba(tmp)[:, 1][0], 3)

0.776

---

### 참고

In [118]:
DataSet6['POPULAR'].unique()

array([0, 1], dtype=int64)

In [123]:
test = np.array([
    [1, 'a'],
    [2, 'b'],
    [3, 'c']
])
test

array([['1', 'a'],
       ['2', 'b'],
       ['3', 'c']], dtype='<U11')

In [124]:
test[:, 1]

array(['a', 'b', 'c'], dtype='<U11')